<a href="https://colab.research.google.com/github/GianvitoC/Causal_Fairness/blob/main/CausalDiscoveryRecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dowhy


     |████████████████████████████████| 164 kB 4.9 MB/s 
  Attempting uninstall: pydot
    Found existing installation: pydot 1.3.0
    Uninstalling pydot-1.3.0:
      Successfully uninstalled pydot-1.3.0


In [3]:
!pip install lingam

     |████████████████████████████████| 67 kB 2.9 MB/s 
